In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#处理Attition
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder

train = train.drop(['EmployeeNumber','StandardHours'],axis=1)
test = test.drop(['EmployeeNumber','StandardHours'],axis=1)

#对分类特征编码
attr = ['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']

for feature in attr:
    lbe = LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    

In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
param ={'boosting_type':'gbdt',
       'objective':'binary',
       'metric':'auc',
       'learning_rate':0.01,
       'max_depth':15,
       'feature_fraction':0.8,
       'bagging_fraction':0.9,
       'bagging_freq':8,
       'lambda_l1':0.6,
       'lambda_l2':0,
       }
X_train,X_valid,y_train,y_valid=train_test_split(train.drop('Attrition',axis=1),train['Attrition'],test_size=0.2,random_state=42)
train_data = lgb.Dataset(X_train,label=y_train)
valid_data = lgb.Dataset(X_valid,label=y_valid)

model=lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round=1000,early_stopping_rounds=200,verbose_eval=25,categorical_feature=attr)
predict=model.predict(test)

test['Attrition']=predict
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)

C:\Users\Ricardo\Anaconda3\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[25]	training's auc: 0.929095	valid_1's auc: 0.773709
[50]	training's auc: 0.942141	valid_1's auc: 0.7957
[75]	training's auc: 0.956017	valid_1's auc: 0.803885
[100]	training's auc: 0.965924	valid_1's auc: 0.807923
[125]	training's auc: 0.973738	valid_1's auc: 0.815126
[150]	training's auc: 0.978803	valid_1's auc: 0.816763
[175]	training's auc: 0.985064	valid_1's auc: 0.819819
[200]	training's auc: 0.98959	valid_1's auc: 0.821019
[225]	training's auc: 0.992824	valid_1's auc: 0.822111
[250]	training's auc: 0.994279	valid_1's auc: 0.823748
[275]	training's auc: 0.995976	valid_1's auc: 0.823857
[300]	training's auc: 0.996964	valid_1's auc: 0.823857
[325]	training's auc: 0.997925	valid_1's auc: 0.824948
[350]	training's auc: 0.998635	valid_1's auc: 0.828113
[375]	training's auc: 0.998922	valid_1's auc: 0.82724
[400]	training's auc: 0.999255	valid_1's auc: 0.828331
[425]	training's auc: 0.999533	valid_1's auc: 0.831496
[450]	trai

In [3]:
test.reset_index()[['user_id','Attrition']].to_csv('submit_lgb.csv',index=None)